In [11]:
import pandas as pd
import numpy as np

# location processing
# from geotext import GeoText
from geopy import geocoders  
# import geocoder
# import requests

import multiprocessing as mp
from multiprocessing import Pool, Array, Lock
from time import time
from joblib import delayed, Parallel

In [16]:
all_data = pd.read_csv('../complete_data/complete_data_sentiment.csv')
all_data.head(1)

,tweet_id,location,date,likes,retweets,text,index,city,state,is_state,...,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate,People_Tested,Mortality_Rate,polarity_neg,polarity_pos,polarity_compound,polarity_classif,polarity_classif2
0,1.250000e+18,"Pomona, CA",4/4/2020,89921,34665,A day without Hispanics .... this country coul...,1,Pomona,California,False,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0


In [17]:
# returns coordinates given location
def get_coords2(i,loc,gn):
    if i %3 == 0:
        print('location',i,':',loc)
    try:
        location = gn.geocode(loc,exactly_one=False)
        coords = location[0][1]
        # print(coords)
    except:
        coords = (None,None)
    return coords

In [19]:
pool = mp.Pool()
# NUM_CPUS=8
gn = geocoders.GeoNames(username='tylerwu')
unique_locs = all_data.location.unique()
index = range(0,len(unique_locs))
# index[:3]
coord_dict = [pool.apply(get_coords2, args=(i, unique_locs[i], gn)) for i in index[:32]] # mp package
# coord_dict = Parallel(n_jobs=NUM_CPUS)(delayed(get_coords2)(i,unique_locs[i],gn) for i in index[:10]) #joblib package
# coord_dict = {unique_locs[i]:get_coords2(i,unique_locs[i], gn) for i in index[:10]} # old-fashioned
# # pool.close()
coord_dict